In [2]:
import datetime
import functools
import json
import os
import re
import sys
import numpy as np


ModuleNotFoundError: No module named 'numpy'

In [3]:
%pip install numpy

     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.0 kB 640.0 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/61.0 kB 525.1 kB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 540.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/15.5 MB 3.2 MB/s eta 0:00:05
   ---------------------------------------- 0.1/15.5 MB 1.0 MB/s eta 0:00:15
    --------------------------------------- 0.2/15.5 MB 1.6 MB/s eta 0:00:10
    --------------------------------------- 0.3/15.5 MB 1.8 MB/s eta 0:00:09
   - -------------------------------------- 0.4/15.5 MB 1.9 MB/s eta 0:00:08
   - -------------------------------------- 0.5/15.5 MB 1.8 MB/s eta 0:00:09
   - -------------------------------------- 0.6/15.5 MB 1.8 MB/s eta 0:00:09
   - -------------------------------------- 0.6/15.5 MB 1.7 MB/s eta 0:00:09
  

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\f2py.exe' -> 'c:\\Python312\\Scripts\\f2py.exe.deleteme'



In [ ]:
def evaluate_loss(X, y, w, b):  # pylint: disable=invalid-name
    residual = y - (X * w + b)
    return np.linalg.norm(residual) ** 2
def gen_meta_prompt(
    old_value_pairs_set,
    X,  # pylint: disable=invalid-name, unused-argument
    y,  # pylint: disable=unused-argument
    num_input_decimals=5,
    num_output_decimals=5,
    max_num_pairs=100,
):
  """Generate the meta-prompt for optimization.
  Args:
   old_value_pairs_set (set): the set of old (w, b, z) pairs.
   X (np.array): the 1D array of x values.
   y (np.array): the 1D array of y values.
   num_input_decimals (int): the number of decimals for (w, b) in the
     meta-prompt.
   num_output_decimals (int): the number of decimals for z in the meta-prompt.
   max_num_pairs (int): the maximum number of exemplars in the meta-prompt.
  Returns:
    meta_prompt (str): the generated meta-prompt.
  """
  old_value_pairs_set = set(
      [  # pylint: disable=g-complex-comprehension
          (
              np.round(w, num_input_decimals)
              if num_input_decimals > 0
              else int(w),
              np.round(b, num_input_decimals)
              if num_input_decimals > 0
              else int(b),
              np.round(z, num_output_decimals)
              if num_output_decimals > 0
              else int(z),
          )
          for w, b, z in old_value_pairs_set
      ]
  )
  old_value_pairs = list(old_value_pairs_set)
  old_value_pairs = sorted(old_value_pairs, key=lambda x: -x[2])[
      -max_num_pairs:
  ]
  old_value_pairs_substr = ""
  for w, b, z in old_value_pairs:
    old_value_pairs_substr += f"\ninput:\nw={w}, b={b}\nvalue:\n{z}\n"
  meta_prompt = """
Now you will help me minimize a function with two input variables w, b. I have some (w, b) pairs and the function values at those points. The pairs are arranged in descending order based on their function values, where lower values are better.
  """.strip()
  meta_prompt += "\n\n"
  meta_prompt += old_value_pairs_substr.strip()
  meta_prompt += "\n\n"
  # function_analytic_form = ""
  # for xi, yi in zip(X, y):
  #   function_analytic_form += f"({yi:.4f} - ({xi:.4f} * w + b)) ** 2 + "
  # function_analytic_form = function_analytic_form[:-3]
  # meta_prompt += (
  #     "The function has the analytic form f(w, b) ="
  #     f" {function_analytic_form}. When evaluating the value of a (w, b)"
  #     " pair, you should replace the w and b in the analytic form with your"
  #     " values and do the computation."
  # )
  # meta_prompt += "\n\n"
  meta_prompt += """Give me a new (w, b) pair that is different from all pairs above, and has a function value lower than any of the above. Do not write code. The output must end with a pair [w, b], where w and b are numerical values.
  """.strip()
  return meta_prompt
def extract_string_in_square_brackets(input_string):
  raw_result = re.findall(r"\[.*?\]", input_string)
  if raw_result:
    for pair in raw_result[::-1]:
      if "=" not in pair and ("w" in pair or "b" in pair):
        continue
      return pair[1:-1]
    return ""
  else:
    return ""
def parse_output(extracted_output):
  """Parse the extracted output 'w, b' string to np.array([w, b]).
  Args:
    extracted_output (str): the extracted output string, like '1.5, 2.5'.
  Returns:
    parsed_output (np.array): the parsed output in a numpy array, like [1.5,
    2.5].
  """
  if not extracted_output:
    return
  extracted_values = []
  for item in extracted_output.split(","):
    if "=" in item:
      item = item[item.index("=") + 1 :]
    extracted_values.append(item.strip())
  parsed_output = np.array(extracted_values).astype(float)
  return parsed_output